<a href="https://colab.research.google.com/github/yogso/music-generation/blob/master/MusicGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Please upload the midi songs with which you want to feed the Machine Learning models { run: "auto", vertical-output: true, display-mode: "both" }
%cd /content

!apt-get install libasound2-dev swig
!pip install git+https://github.com/vishnubob/python-midi@feature/python3
!wget https://raw.githubusercontent.com/yogso/creacion-musical-automatica/master/Red%20Convolucional/manipulacion_midi.py


from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Reshape, Flatten, LSTM
from keras.models import Model, Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.externals import joblib
import numpy as np
import os
import glob
from tqdm import tqdm
import manipulacion_midi
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers.core import Activation, Dropout
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss

#Comprimir y subir las canciones que se usarán en el entrenamiento en un archivo que tenga por nombre "canciones_entrenamiento.zip"
!rm -rf /content/canciones_entrenamiento || true

%mkdir /content/canciones_entrenamiento
%cd /content/canciones_entrenamiento

from google.colab import files
uploaded = files.upload()

%cd /content

#!unzip canciones_entrenamiento.zip -d /content/canciones_entrenamiento

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libasound2 libasound2-data swig3.0
Suggested packages:
  libasound2-plugins alsa-utils libasound2-doc swig-doc swig-examples
  swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  libasound2 libasound2-data libasound2-dev swig swig3.0
0 upgraded, 5 newly installed, 0 to remove and 5 not upgraded.
Need to get 1,619 kB of archives.
After this operation, 8,444 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libasound2-data all 1.1.3-5ubuntu0.1 [36.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libasound2 amd64 1.1.3-5ubuntu0.1 [359 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libasound2-dev amd64 1.1.3-5ubuntu0.1 [123 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-

Using TensorFlow backend.


/content/canciones_entrenamiento


/content


In [0]:
Model_dropdown = 'Convolutional_Network' #@param ["Feedforward_Network", "Decision_Tree", "Random_Forest", "LSTM_Network", "Convolutional_Network"]

previous_times = 5 #@param {type:"slider", min:5, max:20, step:1}

epoch = 300 #@param {type:"integer"}


longitud_minima=15                 #Tamaño minimo (en tiempos) que deben tener las canciones del conjunto de entrenamiento
carpeta_canciones='canciones_entrenamiento'     #Carpeta donde estan las canciones el entrenamiento
modelo_guardado='mejor_modelo.hdf5'  #Nombre del archivo donde se guardara el modelo entrenado

#Elimina los tiempos vacios antes de todas las canciones
def eliminar_zeros_iniciales(arr):
  i=0
  ceros=np.zeros_like(arr[0])
  tamano=np.size(arr,0)
  while np.array_equal(arr[i],ceros) and tamano!=1:
    arr=np.delete(arr,i, axis=0)
    tamano=tamano-1
  return arr

def leer_canciones(ruta):
  print('Loading songs...')
  archivos = glob.glob('{}/*.mid*'.format(ruta))
  archivos = np.array(np.sort(archivos))
  archivos = archivos.tolist()
  global canciones
  global numero_cancion
  canciones = []
  numero_cancion=0
  for f in tqdm(archivos):
    #print(f)
    try:
      cancion = np.array(manipulacion_midi.midiToNoteStateMatrix(f))			#Convierte los archivos .mid a matrices 
      cancion=eliminar_zeros_iniciales(cancion)
      tam = np.array(cancion).shape[0]
      print(tam)
      if tam > longitud_minima:
        if(numero_cancion==0):
          canciones=cancion
          numero_cancion=numero_cancion+1
        else:
          canciones= np.concatenate([canciones, cancion], axis=0)
          numero_cancion=numero_cancion+1
    except Exception as e:
      raise e
  return canciones

#Convierte el arreglo 2D de entradas en un arreglo 3D para entrenar el sistema
def dimensionar(w):
  for t, val in enumerate(w):
    u[0, t] = val
  return u


  #Cargar canciones
canciones = leer_canciones(carpeta_canciones)
print("{} songs were loaded".format(numero_cancion))
    

#Definir modelo a utilizar

def FeedforwardNetworkImpl():
  
  #Dividir las canciones en dos arreglos, uno para los tiempos previos y otro para tiempo de salida  

  sentences=[]
  next_values=[]
  for i in range(0, len(canciones) - previous_times):
    sentences.append(canciones[i: i + previous_times])
    next_values.append(canciones[i + previous_times])
  
  X = np.zeros((len(sentences), previous_times*156), dtype=np.int32)
  for i, sentence in enumerate(sentences):
    X[i] = sentence.ravel() 
  y = np.asarray(next_values)
  
  model = Sequential()
  model.add(Dense(400, activation='relu', input_dim=(previous_times*156)))
  model.add(Dropout(0.1))
  model.add(Dense(400, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(400, activation='relu'))
  model.add(Dense(156))
  model.add(Activation('sigmoid'))
  
  model.summary()

  #Define como se guarda el modelo

  model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['binary_crossentropy'])
  checkpoint = ModelCheckpoint(modelo_guardado, monitor='binary_crossentropy', verbose=1, save_best_only=True, mode='min')
  callbacks_list = [checkpoint]

  #Entrenar modelo

  model.fit(X, y, epochs=epoch, batch_size=2000, callbacks=callbacks_list)

  print ("The model has been successfully trained")

  #Evaluar modelo
  print("The error obtained with the model is: {}". format(model.evaluate(X, y, batch_size=200, verbose=1)))
  
  return model

  
def DecisionTreeImpl():
  
  X=[]
  y=[]
  for i in range(0, len(canciones) - previous_times):
    X.append(np.ravel(canciones[i: i + previous_times]))
    y.append(canciones[i + previous_times])
  
  profundidad_arbol = 100

  #Entrenamiento
  model = tree.DecisionTreeClassifier(max_depth=profundidad_arbol)
  model = model.fit(X, y)

  print ("The model has been successfully trained")

  #Evaluar modelo
  print("The error obtained with the model is: {}". format(log_loss(np.asarray(y), np.asarray(model.predict(X)))))
  
  return model

  
def RandomForestImpl():
  
  X=[]
  y=[]
  for i in range(0, len(canciones) - previous_times):
    X.append(np.ravel(canciones[i: i + previous_times]))
    y.append(canciones[i + previous_times])
  
  profundidad_arbol=200                     #Profundidad máxima de cada árbol individual
  cantidad_arboles=15                       #Árboles que se crearan en el bosque

  model = RandomForestClassifier(max_depth=profundidad_arbol, n_estimators=cantidad_arboles)
  model = model.fit(X, y)

  print ("The model has been successfully trained")

  #Evaluar modelo
  print("The error obtained with the model is: {}". format(log_loss(np.asarray(y), np.asarray(model.predict(X)))))
  
  return model
  

def LSTMNetworkImpl():
  
  #Dividir las canciones en dos arreglos, uno para los tiempos previos y otro para tiempo de salida  

  sentences=[]
  next_values=[]
  for i in range(0, len(canciones) - previous_times):
    sentences.append(canciones[i: i + previous_times])
    next_values.append(canciones[i + previous_times])
    
  #Convertir conjuntos de tiempos previos en un arreglo tridimensional 

  X = np.zeros((len(sentences), previous_times, 156), dtype=np.int32)
  for i, sentence in enumerate(sentences):
    for t, val in enumerate(sentence):
      X[i, t] = val
  y = np.asarray(next_values)
  
  #Definir la red neuronal 
  
  len(canciones)

  model = Sequential()
  model.add(LSTM(400, activation='relu', input_shape=(previous_times, 156), return_sequences=True))
  model.add(Dropout(0.1))
  model.add(LSTM(400, activation='relu', return_sequences=True))
  model.add(Dropout(0.1))
  model.add(LSTM(400, return_sequences=False))
  model.add(Dense(156))
  model.add(Activation('sigmoid'))

  #Define como se guarda el modelo

  model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['binary_crossentropy'])
  checkpoint = ModelCheckpoint(modelo_guardado, monitor='binary_crossentropy', verbose=1, save_best_only=True, mode='min')
  callbacks_list = [checkpoint]

  #Entrenar modelo

  model.fit(X, y, epochs=epoch, batch_size=200, callbacks=callbacks_list)

  print ("The model has been successfully trained")

  #Evaluar modelo
  print("The error obtained with the model is: {}". format(model.evaluate(X, y, batch_size=200, verbose=1)))
  
  return model
  
def ConvolutionalNetworkImpl(): 
  
  #Dividir las canciones en dos arreglos, uno para los tiempos previos y otro para tiempo de salida  

  sentences=[]
  next_values=[]
  for i in range(0, len(canciones) - previous_times):
    sentences.append(canciones[i: i + previous_times])
    next_values.append(canciones[i + previous_times])
    
  
  #Convertir conjuntos de tiempos previos en un arreglo tridimensional 

  X = np.zeros((len(sentences), previous_times, 156), dtype=np.int32)
  for i, sentence in enumerate(sentences):
    for t, val in enumerate(sentence):
      X[i, t] = val
  y = np.asarray(next_values)

  #Definir la red neuronal 
  filtros=40

  model = Sequential()
  model.add(Reshape((previous_times, 156, 1), input_shape=(previous_times, 156)))
  model.add(Conv2D(kernel_size=4, filters=filtros, strides=1, padding='same', activation='relu', data_format='channels_last'))
  model.add(Conv2D(kernel_size=4, filters=filtros, strides=1, padding='same', activation='relu', data_format='channels_last'))
  model.add(Conv2D(kernel_size=4, filters=filtros, strides=1, padding='same', activation='relu', data_format='channels_last'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Flatten())
  model.add(Dense(600))
  model.add(Dense(156))
  model.add(Activation('sigmoid', name='sigmoid'))

  #Define como se guarda el modelo
  model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics=['categorical_crossentropy'])
  checkpoint = ModelCheckpoint(modelo_guardado, monitor='binary_crossentropy', verbose=1, save_best_only=True, mode='min')
  callbacks_list = [checkpoint]

  #Entrenar modelo
  model.fit(X, y, epochs=epoch, batch_size=1000, callbacks=callbacks_list)

  print ("The model has been successfully trained")

  #Evaluar modelo
  print("The error obtained with the model is: {}". format(model.evaluate(X, y, batch_size=200, verbose=1)))

  return model

op_models = {"Feedforward_Network" : FeedforwardNetworkImpl,
           "Decision_Tree" : DecisionTreeImpl, 
           "Random_Forest" : RandomForestImpl, 
           "LSTM_Network" : LSTMNetworkImpl, 
           "Convolutional_Network" : ConvolutionalNetworkImpl
          }

func_model = op_models.get(Model_dropdown, 'Feedforward_Network')

model = func_model()



In [0]:
!ls /content/canciones_generadas/

In [0]:
#@title New songs generation

num_songs = 10 #@param {type:"integer"}

#Comprimir y subir las canciones que se usarán en el entrenamiento en un archivo que tenga por nombre "canciones_entrenamiento.zip"
!rm -rf /content/canciones_generadas || true

%mkdir /content/canciones_generadas

if Model_dropdown == "Decision_Tree" or Model_dropdown == "Random_Forest":
  
  w = np.random.choice([0, 1], size=(previous_times,156),  p=[0.95,0.05])
  plt.imshow(w)
  
    #Generar nueva cancion

  tamano_cancion=200         #Numero de tiempos que tendra la cancion generada

  
  for num_song in tqdm(range (1,num_songs)):
  
    u = np.zeros((1, previous_times, 156), dtype=np.int32)
    tamano_cancion=tamano_cancion+previous_times
    cancion_base=w
    ent=np.zeros((tamano_cancion,156))
    ent[0:previous_times,:]=cancion_base
    for i in tqdm(range (0,tamano_cancion-previous_times)):			#Creacion de la cancion
      nueva_fila = model.predict(np.reshape(ent[i:i+previous_times,:].ravel(),(1,previous_times*156)))
      if (np.all(np.round_(nueva_fila)==0)):
        nueva_fila[0,np.argsort(-nueva_fila)[0,:4]]=1
      ent[i+previous_times] = nueva_fila
      ent=np.round_(ent)

    ent=ent[previous_times:tamano_cancion+previous_times,:]

    #Guardar archivo midi
    manipulacion_midi.noteStateMatrixToMidi(ent, "/content/canciones_generadas/cancion_generada{}".format(num_song))
    
    
else:

  #Generar aleatoriamente entrada inicial

  w = np.random.choice([0, 1], size=(previous_times,156),  p=[0.95,0.05])
  plt.imshow(w)

  #Generar nueva cancion

  tamano_cancion=200         #Numero de tiempos que tendra la cancion generada

  for num_song in tqdm(range (1,num_songs)):

    u = np.zeros((1, previous_times, 156), dtype=np.int32)
    tamano_cancion=tamano_cancion+previous_times
    cancion_base=w
    ent=np.zeros((tamano_cancion,156))
    ent[0:previous_times,:]=cancion_base
    for i in range (0,tamano_cancion-previous_times):			#Creacion de la cancion
      nueva_fila = model.predict(dimensionar(ent[i:i+previous_times,:]), batch_size=1, verbose=1)
      if (np.all(np.round_(nueva_fila)==0)):
        nueva_fila[0,np.argsort(-nueva_fila)[0,:4]]=1
      ent[i+previous_times] = nueva_fila
      ent=np.round_(ent)

    ent=ent[previous_times:tamano_cancion+previous_times,:]

    #Guardar archivo midi
    manipulacion_midi.noteStateMatrixToMidi(ent, "/content/canciones_generadas/cancion_generada{}".format(num_song))
    
    
for f in range (1,num_songs):
  files.download("/content/canciones_generadas/cancion_generada{}.mid".format(f))

In [0]:
!ls

canciones_entrenamiento  manipulacion_midi.py  __pycache__  sample_data
